# CRISP-DM Bullshit here

### Import librairies

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functions import *
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.preprocessing import OneHotEncoder
from IPython.display import Image  
from pydotplus import graph_from_dot_data
from pprint import pprint

%matplotlib inline

### Read csv file and preview file

In [24]:
df = pd.read_csv("Poker.csv", encoding='latin-1')
df.head()

,Player Name,Site,Hands,Net Won,VP$IP,PFR,3Bet,Postflop Agg%,W$WSF%,WTSD%,...,Raise River CBet,Squeeze,Raise Two Raisers,Call Two Raisers,vs 3Bet Fold,vs 3Bet Call,vs 3Bet Raise,vs 4Bet Fold,vs 4Bet Call,vs 4Bet Raise
0,LocSta,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,.PARTOUCHE.,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733
2,julien8082,22,110413,20157.79,0.277494,0.239465,0.077709,0.374164,0.475568,0.241437,...,0.190751,0.081782,0.044171,0.005646,0.656114,0.229410,0.115623,0.471349,0.173752,0.354898
3,PapiersSVP,0,103957,17549.39,0.257664,0.221630,0.109587,0.345363,0.467605,0.300238,...,0.099010,0.107238,0.051926,0.016876,0.541318,0.270994,0.188692,0.343150,0.281690,0.375160
4,NextPlease,22,99879,7326.03,0.260485,0.212327,0.082384,0.328379,0.461007,0.263755,...,0.147826,0.067406,0.023245,0.028610,0.596575,0.290834,0.114494,0.520629,0.108055,0.371316


In [25]:
print("Before filtering")
winners = len(df[df["Net Won"] > 0])
losers = len(df[df["Net Won"] <= 0])
print(winners, "winners")
print(losers, "losers")
print("Winners %", winners / (winners + losers))

Before filtering
3085 winners
5737 losers
Winners % 0.3496939469508048


In [26]:
# df_filtered = df[df['Hands'] > 500]
# print("After filtering")
# winners_f = len(df_filtered[df_filtered["Net Won"] > 0])
# losers_f = len(df_filtered[df_filtered["Net Won"] <= 0])
# print(winners_f, " winners")
# print(losers_f, "losers")
# print("Winners %", winners_f / (winners_f + losers_f))

In [27]:
pprint(list(df.columns))

['Player Name',
 'Site',
 'Hands',
 'Net Won',
 'VP$IP',
 'PFR',
 '3Bet',
 'Postflop\nAgg%',
 'W$WSF%',
 'WTSD%',
 'Won $\nat SD',
 'Flop\nCBet%',
 'Turn\nCBet%',
 'River\nCBet%',
 'Fold to\nFlop Cbet',
 'Fold to\nTurn CBet',
 'Fold to\nRiver CBet',
 'Raise\nFlop Cbet',
 'Raise\nTurn CBet',
 'Raise\nRiver CBet',
 'Squeeze',
 'Raise Two\nRaisers',
 'Call Two\nRaisers',
 'vs 3Bet\nFold',
 'vs 3Bet\nCall',
 'vs 3Bet\nRaise',
 'vs 4Bet\nFold',
 'vs 4Bet\nCall',
 'vs 4Bet\nRaise']


### Rename columns for more clarity (replacing \n by spaces)

In [28]:
df.columns = df.columns.str.replace('\n',' ')
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns

In [29]:
df.head()

,Player_Name,Site,Hands,Net_Won,VP$IP,PFR,3Bet,Postflop_Agg%,W$WSF%,WTSD%,...,Raise_River_CBet,Squeeze,Raise_Two_Raisers,Call_Two_Raisers,vs_3Bet_Fold,vs_3Bet_Call,vs_3Bet_Raise,vs_4Bet_Fold,vs_4Bet_Call,vs_4Bet_Raise
0,LocSta,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,.PARTOUCHE.,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733
2,julien8082,22,110413,20157.79,0.277494,0.239465,0.077709,0.374164,0.475568,0.241437,...,0.190751,0.081782,0.044171,0.005646,0.656114,0.229410,0.115623,0.471349,0.173752,0.354898
3,PapiersSVP,0,103957,17549.39,0.257664,0.221630,0.109587,0.345363,0.467605,0.300238,...,0.099010,0.107238,0.051926,0.016876,0.541318,0.270994,0.188692,0.343150,0.281690,0.375160
4,NextPlease,22,99879,7326.03,0.260485,0.212327,0.082384,0.328379,0.461007,0.263755,...,0.147826,0.067406,0.023245,0.028610,0.596575,0.290834,0.114494,0.520629,0.108055,0.371316


### Build the target based on Net Won vs Hands Played

In [30]:
df["target"] = df["Net_Won"] > 0

### Drop useless columns

In [31]:
df.drop(['Player_Name', 'Site', "Hands", "Net_Won"], axis=1, inplace=True)

## Pre-processing data

#### Check data types

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8822 entries, 0 to 8821
Data columns (total 26 columns):
VP$IP                 8822 non-null float64
PFR                   8822 non-null float64
3Bet                  8822 non-null float64
Postflop_Agg%         8822 non-null float64
W$WSF%                8822 non-null float64
WTSD%                 8822 non-null float64
Won_$_at_SD           8822 non-null float64
Flop_CBet%            8822 non-null float64
Turn_CBet%            8822 non-null float64
River_CBet%           8822 non-null float64
Fold_to_Flop_Cbet     8822 non-null float64
Fold_to_Turn_CBet     8822 non-null float64
Fold_to_River_CBet    8822 non-null float64
Raise_Flop_Cbet       8822 non-null float64
Raise_Turn_CBet       8822 non-null float64
Raise_River_CBet      8822 non-null float64
Squeeze               8822 non-null float64
Raise_Two_Raisers     8822 non-null float64
Call_Two_Raisers      8822 non-null float64
vs_3Bet_Fold          8822 non-null float64
vs_3Bet_Call 

All our predictors are floats (percentages), there are no missing values and our target is a boolean

#TODO: Check for number of unique values in the last 15 columns and maybe drop the ones that don't have a lot of different values meaning there were not enough occurences

### Check for class imbalance

In [43]:
winners = len(df[df["target"] == True])
losers = len(df[df["target"] == False])
print(winners, "winners")
print(losers, "losers")
print("Winners %", winners / (winners + losers))
print("Losers %", 1 - (winners / (winners + losers)))

3085 winners
5737 losers
Winners % 0.3496939469508048
Losers % 0.6503060530491952


The class imbalance isn't very strong, therefore we can move on with this data

## Interactions

In [ ]:
# Todo: Build interaction features

## Polynomials

In [ ]:
# Todo: Build polynomials features

### Split the data between train, validation and test datasets

In [ ]:
labels = df["target"]
df = df.drop(["target"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.20, random_state=42)

### Scale using min max scaling ?

In [ ]:
# # Instantiate StandardScaler
# scaler = StandardScaler()
# # MinMaxScaler(feature_range=(0, 1), copy=True)
# # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

# # Transform the training and test sets
# scaled_data_train = scaler.fit_transform(X_train)
# scaled_data_test = scaler.transform(X_test)

# # Convert into a DataFrame
# scaled_df_train = pd.DataFrame(scaled_data_train, columns=one_hot_df.columns)
# scaled_df_train.head()

## Pipelines

In [ ]:
# # Construct some pipelines
# pipe_svm = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', svm.SVC(random_state=123))])
        
# pipe_tree = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', tree.DecisionTreeClassifier(random_state=123))])

# pipe_rf = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', RandomForestClassifier(random_state=123))])

# # List of pipelines, List of pipeline names
# pipelines = [pipe_svm, pipe_tree, pipe_rf]
# pipeline_names = ['Support Vector Machine','Decision Tree','Random Forest']

# # Loop to fit each of the three pipelines
# for pipe in pipelines:
#     print(pipe)
#     pipe.fit(X_train, y_train)

# # Compare accuracies
# for index, val in enumerate(pipelines):
#     print('%s pipeline test accuracy: %.3f' % (pipeline_names[index], val.score(X_test, y_test)))

## Pipelines with gridsearch

In [ ]:
# # ⏰ This cell may take a long time to run!
# # Construct pipeline
# pipe_rf = Pipeline([('pca', PCA(n_components=27)),
#             ('clf', RandomForestClassifier(random_state = 123))])

# # Set grid search params
# param_grid_forest = [ 
#   {'clf__n_estimators': [120],
#    'clf__criterion': ['entropy', 'gini'], 
#    'clf__max_depth': [4, 5, 6],  
#    'clf__min_samples_leaf':[0.05 ,0.1, 0.2],  
#    'clf__min_samples_split':[0.05 ,0.1, 0.2]
#   }
# ]

# # Construct grid search
# gs_rf = GridSearchCV(estimator=pipe_rf,
#             param_grid=param_grid_forest,
#             scoring='accuracy',
#             cv=3, verbose=2, return_train_score = True)

# # Fit using grid search
# gs_rf.fit(X_train, y_train)

# # Best accuracy
# print('Best accuracy: %.3f' % gs_rf.best_score_)

# # Best params
# print('\nBest params:\n', gs_rf.best_params_)

## Running logistic regression with different thresholds

## Run KNNs

In [ ]:
# Instantiate KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit the classifier
clf.fit(scaled_data_train, y_train)

# Predict on the test set
test_preds = clf.predict(scaled_data_test)

# evaluation metrics for the test predictions stored in test_preds, and the corresponding labels in y_test.
print_metrics(y_test, test_preds)

# Finding the best K
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25)

For further tuning, you can use scikit-learn's built-in GridSearch() to perform a similar exhaustive check of hyperparameter combinations and fine tune model performance. For a full list of model parameters, see the sklearn documentation !

## Run SVMs

https://github.com/learn-co-students/dsc-building-an-svm-using-scikit-learn-lab-london-ds-111819/tree/solution

Kernel Trick
https://github.com/learn-co-students/dsc-kernels-in-scikit-learn-lab-london-ds-111819/tree/solution

## Run Decision Trees

In [ ]:
# Create the classifier, fit it on the training data and make predictions on the test set
# clf = DecisionTreeClassifier(criterion='entropy')

# clf.fit(X_train, y_train)

## Run Clustering

## Run Ensemble Methods

# Conclusions